<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.

## Setup

In [8]:
import sqlite3 as lite
import sys
import requests
import pandas as pd
import datetime
import json

## Class & function definitions


In [9]:
class Coin:
    def __init__(self, symbol, comparison_symbol, limit, aggregate):
        self.symbol = symbol
        self.comparison_symbol = comparison_symbol
        self.limit = limit
        self.aggregate = aggregate
        
    def fetch_marketdata(self):    
        url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
                .format(self.symbol.upper(), self.comparison_symbol.upper(), self.limit, self.aggregate)       
        page = requests.get(url)
        data = page.json()['Data']
        df = pd.DataFrame(data)
        df['mid'] = df[["high", "low"]].mean(axis=1)
        df['timestamp'] = [datetime.datetime.fromtimestamp(d).strftime('%d-%m-%Y %H:%M:%S') for d in df.time]
        return df

    
    
def write_to_db(market_data):
    con = None
    con = lite.connect('crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        cur = con.cursor()
        cur.execute("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol))
        rows = cur.fetchall()
        latest_db_entry = list(rows[0])
        latest_db_entry = int(latest_db_entry[0])
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])

## Fetch data & store in db

In [10]:
fh = open("tracked_coins.json", 'r')
coinlist = json.load(fh)
coin_instances = [Coin(coinlist[coin], 'BTC','1','1') for coin in range(len(coinlist))] # Make objects. Take all coins in coinlist, convert to BTC, poll 1 entry and aggregate per 1-hour period.

#Get data
for coin in range(len(coin_instances)):
    try:
        market_data = coin_instances[coin].fetch_marketdata() # Get coindata
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        write_to_db(market_data)
    except Exception as e:
        print("Skipping coin", coin_instances[coin], ", Error: ")
        print(e)